### Manipulation of PEAKS de novo results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >50% ALC

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/307_TROCAS7_Fusion_Apr2021_DENOVO_181/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/307_TROCAS7_Fusion_Apr2021_DENOVO_181


In [3]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks306 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/307_TROCAS7_Fusion_Apr2021_DENOVO_181/307_TROCAS7_Fusion_Apr2021_DENOVO_181_DN50.csv")

print("# redundant Peaks peptides >50% ALC in combined dataframe:", len(peaks306))

print(peaks306.columns)

# These columns mess things up- get rid of them

del peaks306['Fraction']
del peaks306['Scan']
del peaks306['Source File']
del peaks306['Tag Length']
del peaks306['PTM']
del peaks306['tag (>=0%)']
del peaks306['mode']
del peaks306['local confidence (%)']


columns = ['Peptide', 'ALC', 'length', 'm/z', 'z', 'RT', 'Area',
       'Mass', 'ppm']

peaks306.columns = columns

mean_len = peaks306['length'].mean()
print(mean_len)

# look at the dataframe
peaks306.head()

# redundant Peaks peptides >50% ALC in combined dataframe: 178
Index(['Fraction', 'Scan', 'Source File', 'Peptide', 'Tag Length', 'ALC (%)',
       'length', 'm/z', 'z', 'RT', 'Area', 'Mass', 'ppm', 'PTM',
       'local confidence (%)', 'tag (>=0%)', 'mode'],
      dtype='object')
10.112359550561798


,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm
0,LSSPATLNSR,98,10,523.2872,2,47.14,57900000.0,1044.5564,3.2
1,REKM(+15.99)SSR,97,7,455.2311,2,30.39,111000.0,908.4498,-2.2
2,SSPATLNDK,95,9,466.7441,2,27.67,93000.0,931.4611,13.6
3,RSLQVR,94,6,379.7358,2,22.72,103000.0,757.4559,1.5
4,LSSPATLNSR,94,10,523.2864,2,46.64,57900000.0,1044.5564,1.7


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine: 15.9949 M
    variable deamidation of asparagine, glumatine: 0.984016 NQ

We'll then write this manipulated dataframe to a new file.

In [4]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks306['A'] = peaks306['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks306['C'] = peaks306['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks306['D'] = peaks306['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks306['E'] = peaks306['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks306['F'] = peaks306['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks306['G'] = peaks306['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks306['H'] = peaks306['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks306 output, there will be no isoleucines (they're lumped in with leucines)
peaks306['I'] = peaks306['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks306['K'] = peaks306['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks306['L'] = peaks306['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks306['M'] = peaks306['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks306['N'] = peaks306['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks306['P'] = peaks306['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks306['Q'] = peaks306['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks306['R'] = peaks306['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks306['S'] = peaks306['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks306['T'] = peaks306['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks306['V'] = peaks306['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks306['W'] = peaks306['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks306['Y'] = peaks306['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks306['c-carb'] = peaks306['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks306['m-oxid'] = peaks306['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks306['n-deam'] = peaks306['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks306['q-deam'] = peaks306['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaks306['stripped_peptide'] = peaks306['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks306['stripped_length'] = peaks306['stripped_peptide'].apply(len)

# total the number of modifications in sequence
peaks306['ptm-total'] = peaks306['c-carb'] + peaks306['m-oxid'] + peaks306['n-deam'] + peaks306['q-deam']

# calculate NAAF numerator for each peptide k
peaks306['NAAF_num.'] = peaks306['Area'] / peaks306['stripped_length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks306.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/306_CV_T24_GF_DN/306C_CV_T24_GF_DN50.csv")

# check out the results
peaks306.head()

,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm,A,...,W,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,ptm-total,NAAF_num.
0,LSSPATLNSR,98,10,523.2872,2,47.14,57900000.0,1044.5564,3.2,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,5.790000e+06
1,REKM(+15.99)SSR,97,7,455.2311,2,30.39,111000.0,908.4498,-2.2,0,...,0,0,0,1,0,0,REKMSSR,7,1,1.585714e+04
2,SSPATLNDK,95,9,466.7441,2,27.67,93000.0,931.4611,13.6,1,...,0,0,0,0,0,0,SSPATLNDK,9,0,1.033333e+04
3,RSLQVR,94,6,379.7358,2,22.72,103000.0,757.4559,1.5,0,...,0,0,0,0,0,0,RSLQVR,6,0,1.716667e+04
4,LSSPATLNSR,94,10,523.2864,2,46.64,57900000.0,1044.5564,1.7,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,5.790000e+06


In [5]:
# keep only stripped peptide I/L and NAAF
dn_306 = peaks306[['stripped_peptide', 'Area', 'NAAF_num.']]

dn_306.set_index('stripped_peptide')

# write modified dataframe to new txt file
dn_306.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/306C_CV_T24_GF_DN50_ILnaafs.csv")

dn_306.head()

,stripped_peptide,Area,NAAF_num.
0,LSSPATLNSR,57900000.0,5.790000e+06
1,REKMSSR,111000.0,1.585714e+04
2,SSPATLNDK,93000.0,1.033333e+04
3,RSLQVR,103000.0,1.716667e+04
4,LSSPATLNSR,57900000.0,5.790000e+06


In [6]:
# made a new dataframe that contains the suMN of certain columns 
# in the stripped peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks306['A'].sum(),
        'C': peaks306['C'].sum(),
        'D': peaks306['D'].sum(),
        'E': peaks306['E'].sum(),
        'F': peaks306['F'].sum(),
        'G': peaks306['G'].sum(),
        'H': peaks306['H'].sum(),
        'I': peaks306['I'].sum(),
        'K': peaks306['K'].sum(),
        'L': peaks306['L'].sum(),
        'M': peaks306['M'].sum(),
        'N': peaks306['N'].sum(),
        'P': peaks306['P'].sum(),
        'Q': peaks306['Q'].sum(),
        'R': peaks306['R'].sum(),
        'S': peaks306['S'].sum(),
        'T': peaks306['T'].sum(),
        'V': peaks306['V'].sum(),
        'W': peaks306['W'].sum(),
        'Y': peaks306['Y'].sum(),
        'c-carb': peaks306['c-carb'].sum(),
        'm-oxid': peaks306['m-oxid'].sum(),
        'n-deam': peaks306['n-deam'].sum(),
        'q-deam': peaks306['q-deam'].sum(),
        'Total area': peaks306['Area'].sum(),
        'Total length': peaks306['stripped_length'].sum()
       }

totalpeaks306 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'n-deam', 'q-deam', \
                                            'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks306['% C w/ carb'] = totalpeaks306['c-carb'] / totalpeaks306['C'] 

# calculate percentage of M's that are oxidized
totalpeaks306['% M w/ oxid'] = totalpeaks306['m-oxid'] / totalpeaks306['M'] 

# calculate percentage of N's that are deamidated
totalpeaks306['% N w/ deam'] = totalpeaks306['n-deam'] / totalpeaks306['N'] 

# calculate percentage of N's that are deamidated
totalpeaks306['% Q w/ deam'] = totalpeaks306['q-deam'] / totalpeaks306['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks306['NAAF denom.'] = totalpeaks306['Total area'] / totalpeaks306['Total length']

# write modified dataframe to new txt file
totalpeaks306.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/306_CV_T24_GF_DN/306C_CV_T24_GF_DN50_totals.csv")

totalpeaks306.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,110,46,36,52,54,92,134,0,159,227,...,26,14,0,1.150501e+09,1682,1.0,0.262626,0.2,0.0,684007.462961


In [7]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF50 = 684007.462961

# use NAAF >50% ALC to get NAAF factor
peaks306['NAAF factor'] = (peaks306['NAAF_num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksAAPTM_306 = peaks306[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'n-deam', 'q-deam']].copy()

# multiply the NAAF50 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_306['A-NAAF50'] = peaksAAPTM_306['A'] * peaks306['NAAF factor']
peaksAAPTM_306['C-NAAF50'] = peaksAAPTM_306['C'] * peaks306['NAAF factor']
peaksAAPTM_306['D-NAAF50'] = peaksAAPTM_306['D'] * peaks306['NAAF factor']
peaksAAPTM_306['E-NAAF50'] = peaksAAPTM_306['E'] * peaks306['NAAF factor']
peaksAAPTM_306['F-NAAF50'] = peaksAAPTM_306['F'] * peaks306['NAAF factor']
peaksAAPTM_306['G-NAAF50'] = peaksAAPTM_306['G'] * peaks306['NAAF factor']
peaksAAPTM_306['H-NAAF50'] = peaksAAPTM_306['H'] * peaks306['NAAF factor']
peaksAAPTM_306['I-NAAF50'] = peaksAAPTM_306['I'] * peaks306['NAAF factor']
peaksAAPTM_306['K-NAAF50'] = peaksAAPTM_306['K'] * peaks306['NAAF factor']
peaksAAPTM_306['L-NAAF50'] = peaksAAPTM_306['L'] * peaks306['NAAF factor']
peaksAAPTM_306['M-NAAF50'] = peaksAAPTM_306['M'] * peaks306['NAAF factor']
peaksAAPTM_306['N-NAAF50'] = peaksAAPTM_306['N'] * peaks306['NAAF factor']
peaksAAPTM_306['P-NAAF50'] = peaksAAPTM_306['P'] * peaks306['NAAF factor']
peaksAAPTM_306['Q-NAAF50'] = peaksAAPTM_306['Q'] * peaks306['NAAF factor']
peaksAAPTM_306['R-NAAF50'] = peaksAAPTM_306['R'] * peaks306['NAAF factor']
peaksAAPTM_306['S-NAAF50'] = peaksAAPTM_306['S'] * peaks306['NAAF factor']
peaksAAPTM_306['T-NAAF50'] = peaksAAPTM_306['T'] * peaks306['NAAF factor']
peaksAAPTM_306['V-NAAF50'] = peaksAAPTM_306['V'] * peaks306['NAAF factor']
peaksAAPTM_306['W-NAAF50'] = peaksAAPTM_306['W'] * peaks306['NAAF factor']
peaksAAPTM_306['Y-NAAF50'] = peaksAAPTM_306['Y'] * peaks306['NAAF factor']

# multiply the NAAF50 factor by the PTMN normalize its abundance by peak area and peptide length

peaksAAPTM_306['ccarb-NAAF50'] = peaksAAPTM_306['c-carb'] * peaksAAPTM_306['NAAF factor']
peaksAAPTM_306['moxid-NAAF50'] = peaksAAPTM_306['m-oxid'] * peaksAAPTM_306['NAAF factor']
peaksAAPTM_306['ndeam-NAAF50'] = peaksAAPTM_306['n-deam'] * peaksAAPTM_306['NAAF factor']
peaksAAPTM_306['qdeam-NAAF50'] = peaksAAPTM_306['q-deam'] * peaksAAPTM_306['NAAF factor']


# write the dataframe to a new csv
peaksAAPTM_306.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/306_CV_T24_GF_DN/306C_CV_T24_GF_DN50_naaf.csv")

peaksAAPTM_306.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,K,...,R-NAAF50,S-NAAF50,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50
0,LSSPATLNSR,8.464820,1,0,0,0,0,0,0,0,...,8.464820,25.394460,8.464820,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
1,REKMSSR,0.023183,0,0,0,1,0,0,0,1,...,0.046365,0.046365,0.000000,0.000000,0.0,0.0,0.0,0.023183,0.0,0.0
2,SSPATLNDK,0.015107,1,0,1,0,0,0,0,1,...,0.000000,0.030214,0.015107,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,RSLQVR,0.025097,0,0,0,0,0,0,0,0,...,0.050194,0.025097,0.000000,0.025097,0.0,0.0,0.0,0.000000,0.0,0.0
4,LSSPATLNSR,8.464820,1,0,0,0,0,0,0,0,...,8.464820,25.394460,8.464820,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [8]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMN

index = ['sample total']

data = {'NAAF': peaksAAPTM_306['NAAF factor'].sum(),
        'A': peaksAAPTM_306['A-NAAF50'].sum(),
        'C': peaksAAPTM_306['C-NAAF50'].sum(),
        'D': peaksAAPTM_306['D-NAAF50'].sum(),
        'E': peaksAAPTM_306['E-NAAF50'].sum(),
        'F': peaksAAPTM_306['F-NAAF50'].sum(),
        'G': peaksAAPTM_306['G-NAAF50'].sum(),
        'H': peaksAAPTM_306['H-NAAF50'].sum(),
        'I': peaksAAPTM_306['I-NAAF50'].sum(),
        'K': peaksAAPTM_306['K-NAAF50'].sum(),
        'L': peaksAAPTM_306['L-NAAF50'].sum(),
        'M': peaksAAPTM_306['M-NAAF50'].sum(),
        'N': peaksAAPTM_306['N-NAAF50'].sum(),
        'P': peaksAAPTM_306['P-NAAF50'].sum(),
        'Q': peaksAAPTM_306['Q-NAAF50'].sum(),
        'R': peaksAAPTM_306['R-NAAF50'].sum(),
        'S': peaksAAPTM_306['S-NAAF50'].sum(),
        'T': peaksAAPTM_306['T-NAAF50'].sum(),
        'V': peaksAAPTM_306['V-NAAF50'].sum(),
        'W': peaksAAPTM_306['W-NAAF50'].sum(),
        'Y': peaksAAPTM_306['Y-NAAF50'].sum(),
        'c-carb': peaksAAPTM_306['ccarb-NAAF50'].sum(),
        'm-oxid': peaksAAPTM_306['moxid-NAAF50'].sum(),
        'n-deam': peaksAAPTM_306['ndeam-NAAF50'].sum(),
        'q-deam': peaksAAPTM_306['qdeam-NAAF50'].sum(),
       }

totalpeaks50_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks50_NAAF['% C w/ carb.'] = totalpeaks50_NAAF['c-carb'] / totalpeaks50_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks50_NAAF['% M w/ oxid'] = totalpeaks50_NAAF['m-oxid'] / totalpeaks50_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% N w/ deam'] = totalpeaks50_NAAF['n-deam'] / totalpeaks50_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% Q w/ deam'] = totalpeaks50_NAAF['q-deam'] / totalpeaks50_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks50_NAAF['NAAF check'] = totalpeaks50_NAAF['NAAF'] / 684007.462961

# write modified dataframe to new txt file, same name + totals
totalpeaks50_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/306_CV_T24_GF_DN/306C_CV_T24_GF_DN50_naaf_totals.csv")

totalpeaks50_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,202.863142,132.135364,6.82014,2.184253,2.846538,3.518906,8.639759,7.462161,0.0,7.828093,...,2.605237,6.82014,2.619312,2.062754,0.0,1.0,0.403527,0.06717,0.0,0.000297


## Export stripped peptides >50% ALC

In [9]:
##### keep only stripped peptide column 
pep50 = peaks306[["stripped_peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep50.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/306_CV_T24_GF_DN/306C_CV_T24_GF_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/306_CV_T24_GF_DN/306C_CV_T24_GF_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/306_CV_T24_GF_DN/306C_CV_T24_GF_DN50_stripped_peptides.fas

# removing redundancy
peaks50dedup = pd.DataFrame.drop_duplicates(pep50)

print("# redundant stripped Peaks peptides >50% ALC", len(pep50))
print("# nonredundant stripped Peaks peptides >50% ALC", len(peaks50dedup))
print("average peptide length Peaks peptides >50% ALC", peaks306['stripped_length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >50% ALC
pep50m = peaks306[["Peptide"]]

# deduplicate
pep50mdedup = pd.DataFrame.drop_duplicates(pep50m)

print("# redundant Peaks peptides >50% ALC", len(pep50m))
print("# nonredundant Peaks peptides", len(pep50mdedup))

# check
pep50.head()

# redundant stripped Peaks peptides >50% ALC 178
# nonredundant stripped Peaks peptides >50% ALC 138
average peptide length Peaks peptides >50% ALC 9.44943820224719
# redundant Peaks peptides >50% ALC 178
# nonredundant Peaks peptides 139


,stripped_peptide
0,LSSPATLNSR
1,REKMSSR
2,SSPATLNDK
3,RSLQVR
4,LSSPATLNSR
